In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [4]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [5]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [6]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [7]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [8]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [9]:
%%sql
SELECT namefirst,namelast,halloffame.playerid,yearid
FROM halloffame
JOIN people
ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


namefirst,namelast,playerid,yearid
Ty,Cobb,cobbty01,1936
Babe,Ruth,ruthba01,1936
Honus,Wagner,wagneho01,1936
Christy,Mathewson,mathech01,1936
Walter,Johnson,johnswa01,1936
Nap,Lajoie,lajoina01,1937
Tris,Speaker,speaktr01,1937
Cy,Young,youngcy01,1937
Morgan,Bulkeley,bulkemo99,1937
Ban,Johnson,johnsba99,1937


In [10]:
%%sql result_1a <<
SELECT namefirst,namelast,halloffame.playerid,yearid
FROM halloffame
JOIN people
ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [11]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [12]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [13]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [14]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=14.250..15.510 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.011..1.006 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=14.021..14.023 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.009..6.690 rows=19370 loops=1)
 Planning Time: 1.332 ms
 Execution Time: 15.642 ms
(10 rows)



In [15]:
inducted_hof_query_str = "SELECT namefirst,namelast,halloffame.playerid,yearid FROM halloffame JOIN people ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=13.622..14.779 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (people.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.011..0.950 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=13.352..13.354 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.014..6.585 rows=19370 loops=1)
 Planning Time: 1.408 ms
 Execution Time: 14.930 ms
(10 rows)



In [16]:
provided_query_cost = 959.06
provided_query_timing = 11.742
your_query_cost = 959.06
your_query_timing = 10.925

In [17]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [18]:
q1b_part2 = ['A', 'C', 'F']

In [19]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [20]:
%%sql
WITH ca_players AS (
SELECT cp.playerid, cp.schoolid
FROM collegeplaying AS cp
JOIN schools AS s
ON cp.schoolid = s.schoolid
AND s.schoolstate = 'CA'
),
hof AS (
SELECT namefirst,namelast,halloffame.playerid,yearid
FROM halloffame
JOIN people
ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'
)
SELECT namefirst, namelast, hof.playerid, schoolid, yearid
FROM hof
JOIN ca_players
ON hof.playerid = ca_players.playerid
ORDER BY yearid, hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
23 rows affected.


namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,usc,1992
Tom,Seaver,seaveto01,cafrecc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [21]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
WITH ca_players AS (
SELECT cp.playerid, cp.schoolid
FROM collegeplaying AS cp
JOIN schools AS s
ON cp.schoolid = s.schoolid
AND s.schoolstate = 'CA'
),
hof AS (
SELECT namefirst,namelast,halloffame.playerid,yearid
FROM halloffame
JOIN people
ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'
)
SELECT namefirst, namelast, hof.playerid, schoolid, yearid
FROM hof
JOIN ca_players
ON hof.playerid = ca_players.playerid
ORDER BY yearid, hof.playerid;
SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [22]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [23]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [24]:
%%sql
SELECT cp.schoolid, hal.yearid, COUNT(*)
FROM collegeplaying AS cp 
JOIN schools AS s ON cp.schoolid = s.schoolid AND s.schoolstate = 'CA'
JOIN halloffame AS hal ON cp.playerid = hal.playerid AND hal.inducted = 'Y'
GROUP BY cp.schoolid, hal.yearid
ORDER BY hal.yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.


schoolid,yearid,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [25]:
%%sql with_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [26]:
%%sql without_view <<
SELECT cp.schoolid, hal.yearid, COUNT(*)
FROM collegeplaying AS cp 
JOIN schools AS s ON cp.schoolid = s.schoolid AND s.schoolstate = 'CA'
JOIN halloffame AS hal ON cp.playerid = hal.playerid AND hal.inducted = 'Y'
GROUP BY cp.schoolid, hal.yearid
ORDER BY hal.yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [27]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [28]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [29]:
with_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                              QUERY PLAN                                                                              
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=525.47..527.39 rows=96 width=20) (actual time=5.808..5.822 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=525.47..525.71 rows=96 width=12) (actual time=5.799..5.805 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=521.11..522.31 rows=96 width=12) (actual time=5.751..5.761 rows=23 loops=1)
               ->  Sort  (cost=521.11..521.35 rows=96 width=257) (actual time=5.749..5.755 rows=23 loops=1)
                     Sort Key: halloffame.yearid, halloffa

In [30]:
without_view_query_str = "SELECT cp.schoolid, hal.yearid, COUNT(*) FROM collegeplaying AS cp  JOIN schools AS s ON cp.schoolid = s.schoolid AND s.schoolstate = 'CA' JOIN halloffame AS hal ON cp.playerid = hal.playerid AND hal.inducted = 'Y' GROUP BY cp.schoolid, hal.yearid ORDER BY hal.yearid"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                     QUERY PLAN                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=488.95..490.87 rows=96 width=20) (actual time=5.824..5.843 rows=13 loops=1)
   Group Key: hal.yearid, cp.schoolid
   ->  Sort  (cost=488.95..489.19 rows=96 width=12) (actual time=5.813..5.818 rows=23 loops=1)
         Sort Key: hal.yearid, cp.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=12) (actual time=5.370..5.726 rows=23 loops=1)
               Hash Cond: ((hal.playerid)::text = (cp.playerid)::text)
               ->  Seq Scan on halloffame hal  (cost=0.00..96.39 rows=323 width=13) (actual time=0.009..0.599 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
                

In [31]:
with_view_cost = 527.39
with_view_timing = 6.538
without_view_cost = 490.87
without_view_timing = 5.923

In [32]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [33]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
WITH ca_players AS (
SELECT cp.playerid, cp.schoolid
FROM collegeplaying AS cp
JOIN schools AS s
ON cp.schoolid = s.schoolid
AND s.schoolstate = 'CA'
),
hof AS (
SELECT namefirst,namelast,halloffame.playerid,yearid
FROM halloffame
JOIN people
ON halloffame.playerid = people.playerid AND halloffame.inducted = 'Y'
)
SELECT namefirst, namelast, hof.playerid, schoolid, yearid
FROM hof
JOIN ca_players
ON hof.playerid = ca_players.playerid
ORDER BY yearid, hof.playerid;
SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [34]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [35]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [36]:
%%sql with_materialized_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [37]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [38]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [39]:
with_materialized_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.083..0.085 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.035..0.040 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.007..0.010 rows=23 loops=1)
 Planning Time: 0.414 ms
 Execution Time: 0.238 ms
(9 rows)



In [40]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.191

In [41]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [42]:
q1e_part4 = ['D', 'J']

In [43]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

A, B, and C are false. Based on the query plans, the query with the view had a higher cost and longer execution time as compared to the query that did not use a view. Conceptually this makes sense because views are computed on demand, so when querying a view, the view must be computed first, then the query is executed. This can cause the optimizer to compute in an order that takes longer than querying without the view, as seen by the different query plans for with the view and without the view. D is correct because a materialized view actually stores the result of the query inside the materialized view, so when the materialized view is queried, all that has to be done is the aggregation and sorting, and not any of the joining and filtering, this is why E, F, and G are all false. J is correct because a virtual view simple stores the definition of the query inside of it, and doesn't actually compute is til it is queried, where the materialized view actually computes it, so computing and storing a query will take longer then simply saving its defintion. This is why H and I are false.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [44]:
query_view_str = "SELECT * FROM inducted_hof_ca"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                     QUERY PLAN                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=530.54..530.78 rows=96 width=33) (actual time=6.554..6.561 rows=23 loops=1)
   Sort Key: halloffame.yearid, halloffame.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=386.71..527.38 rows=96 width=33) (actual time=5.724..6.505 rows=23 loops=1)
         Join Filter: ((halloffame.playerid)::text = (people.playerid)::text)
         ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.670..6.199 rows=23 loops=1)
               Hash Cond: ((halloffame.playerid)::text = (cp.playerid)::text)
               ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.009..0.798 rows=323 loops=1)
                     Filter: ((in

In [45]:
query_view_cost = 530.78
query_view_timing = 8.318

In [46]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [47]:
%%sql result_2b <<
SELECT *
FROM inducted_hof_ca
WHERE yearid > 2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [48]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                              QUERY PLAN                                                                              
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=209.92..209.93 rows=6 width=33) (actual time=0.959..0.962 rows=8 loops=1)
   Sort Key: halloffame.yearid, halloffame.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..209.84 rows=6 width=33) (actual time=0.616..0.921 rows=8 loops=1)
         Join Filter: ((halloffame.playerid)::text = (people.playerid)::text)
         ->  Nested Loop  (cost=0.56..207.24 rows=6 width=30) (actual time=0.582..0.845 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (actual time=0.559..0.731 rows=27 loops=1)
                     ->  Seq Scan on halloffame  (cost=0.00..106.86 rows=21 width=13) (actual t

In [49]:
query_view_with_filter_cost = 209.93
query_view_with_filter_timing = 1.402

In [50]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [51]:
q2c = ['A', 'E']

In [52]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

A and E are ture. Since we are selecting from a view, the optimizer was able to push down the year > 2010 condition into the same step as inducted = 'Y' before the joins are executed in the view. This allows the joins to be done over a smaller subset of the halloffame table, thus reducing the cost to preform those joins because less data is needed to be compaired to, reducing total cost and execution time. This is why the rest of the statements are false.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [53]:
%%sql q3a_join <<
SELECT *
FROM people AS p
INNER JOIN collegeplaying AS cp
ON p.playerid = cp.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [54]:
q3a_query_str = "SELECT * FROM people AS p INNER JOIN collegeplaying AS cp ON p.playerid = cp.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                         QUERY PLAN                                                         
----------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=9.270..19.630 rows=17350 loops=1)
   Hash Cond: ((cp.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying cp  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.006..1.302 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=9.026..9.029 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.011..1.819 rows=19370 loops=1)
 Planning Time: 1.119 ms
 Execution Time: 20.430 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [55]:
q3a = ['C']

In [56]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [57]:
%%sql q3b_join <<
SELECT *
FROM people AS p
INNER JOIN collegeplaying AS cp
ON p.playerid = cp.playerid
ORDER BY cp.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [58]:
q3b_query_str = "SELECT * FROM people AS p INNER JOIN collegeplaying AS cp ON p.playerid = cp.playerid ORDER BY cp.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                         QUERY PLAN                                                                          
-------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.037..18.124 rows=17350 loops=1)
   Merge Cond: ((p.playerid)::text = (cp.playerid)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.014..4.110 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying cp  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.018..2.341 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 0.957 ms
 Execution Time: 18.885 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [59]:
q3b = ['B']

In [60]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [61]:
%%sql q3c_join <<
SELECT *
FROM people AS p1
INNER JOIN people AS p2
ON p1.playerid != p2.playerid
LIMIT 1000;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [62]:
q3c_query_str = "SELECT * FROM people AS p1 INNER JOIN people AS p2 ON p1.playerid != p2.playerid LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.030..0.993 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.029..0.923 rows=1000 loops=1)
         Join Filter: ((p1.playerid)::text <> (p2.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.011..0.011 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.010..0.481 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.085 rows=1001 loops=1)
 Planning Time: 0.829 ms
 Execution Time: 1.162 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [63]:
q3c = ['A']

In [64]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

For the query in 3a, the query optimizer choose to use a hash join. This is because the players hash table with hash key playerid is able to fit into working memory. Thus when scanning the collegeplaying table it just probes the players hash table for the matching join key. This is much faster than sequentially scanning the players table for the matching playerid. Thus it is faster than a merge join or a nested loop join in this situtation. Hash join is usually always selected if both the tables are large, but not too big for one of them to fit into a hash table. Hash joins also get no added benefit from indexes on the join keys, so if there are no indexes on the join keys then hash join is usually the best option.

For the query in 3b, the query optimizer choose to use a merge join. This is because the query also contained a ORDER BY clause on the playerid, and a merge join's output will always be sorted by the join key which in this case is also the playerid, so there is no need to sort after the join. Another benefit to the merge join is that both tables have an index on the playerid join key, so these indexes help speed up the sorting of both tables before they are merged. Merge joins always best when both tables are too large to hash.

For the query in 3c, the query optimizers only option is to use a nested loop join. This is because of the non-equijoin. Hash and Merge join are only applicable when there is an equijoin. In general nested loop joins are better when one of the relations is small. This is because the inner loop will not be executed that many times is the outer relation is small.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [65]:
%%sql result_4a <<
SELECT playerid, AVG(salary)
FROM salaries AS s
NATURAL JOIN appearances AS a
WHERE a.g_batting = 10
GROUP BY playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [66]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [67]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [68]:
result_4a_query_str = "SELECT playerid, AVG(salary) FROM salaries AS s NATURAL JOIN appearances AS a WHERE a.g_batting = 10 GROUP BY playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3635.70..3635.73 rows=1 width=17) (actual time=21.541..21.609 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=3635.70..3635.71 rows=1 width=17) (actual time=21.528..21.538 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.92..3635.69 rows=1 width=17) (actual time=13.273..21.332 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [69]:
result_4a_cost = 3635.73
result_4a_timing = 21.735

In [70]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [71]:
%%sql result_4b <<
CREATE INDEX g_batting_idx ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [72]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2360.13..2360.15 rows=1 width=17) (actual time=10.923..11.001 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=2360.13..2360.14 rows=1 width=17) (actual time=10.909..10.921 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1623.35..2360.12 rows=1 width=17) (actual time=2.766..10.738 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459

In [75]:
result_4b_cost = 2360.15
result_4b_timing = 11.156

In [76]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [77]:
%%sql 
DROP INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [78]:
%%sql result_4c <<
CREATE INDEX salary_idx ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [79]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                   QUERY PLAN                                                                                   
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3635.70..3635.73 rows=1 width=17) (actual time=20.536..20.605 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=3635.70..3635.71 rows=1 width=17) (actual time=20.521..20.531 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2898.92..3635.69 rows=1 width=17) (actual time=12.527..20.294 rows=138 loops=1)
               Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..45

In [80]:
result_4c_cost = 3635.73
result_4c_timing = 20.755

In [81]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [82]:
q4d_part1 = ['D', 'E']

In [83]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

D is true. Based on the query plans, the cost and execution time are both lower with the index on g_batting compaired to without the index on g_batting. This is because we are filtering by g_batting = 10 and adding an index on this attribute allowed the query plan to go directly to the pages where the tuples had g_batting = 10 as opposed to having to do a sequential scan on all the tuples to find which ones have g_batting = 10. Thus A,B, and C are false. 

E is true. Based on the query plan, the cost and execution time are basically the same. This is because we were not filtering by salary, we were still filtering by g_batting, so the index didn't help to find our tuples that satified out filter condition. We were just aggregating on salary, so we still had to go through all the values anyway.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [84]:
%%sql 
DROP INDEX salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [85]:
%%sql result_4ei <<
SELECT playerid, yearid, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_all = 10 AND g_batting = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [86]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [87]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [88]:
result_4ei_query_str = "SELECT playerid, yearid, salary FROM salaries NATURAL JOIN appearances WHERE g_all = 10 AND g_batting = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3287.78 rows=1 width=21) (actual time=18.942..33.044 rows=120 loops=1)
   ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=19 width=20) (actual time=0.018..26.220 rows=1289 loops=1)
         Filter: ((g_all = 10) AND (g_batting = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries  (cost=0.29..8.10 rows=1 width=28) (actual time=0.004..0.004 rows=0 loops=1289)
         Index Cond: ((yearid = appearances.yearid) AND ((teamid)::text = (appearances.teamid)::text) AND ((lgid)::text = (appearances.lgid)::text) AND ((pl

In [89]:
result_4ei_cost = 3300.40
result_4ei_timing = 18.860

In [90]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [91]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [92]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.23..1754.46 rows=1 width=21) (actual time=2.662..5.246 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=17.94..1600.52 rows=19 width=20) (actual time=0.314..2.156 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..17.94 rows=1286 width=0) (actual time=0.194..0.195 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries  (cost=0.

In [93]:
result_4eii_with_index_cost = 1782.99
result_4eii_with_index_timing = 6.309

In [94]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [95]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [96]:
%%sql result_4eiii <<
SELECT playerid, yearid, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_all = 10 OR g_batting = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [97]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [98]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [99]:
result_4eiii_query_str = "SELECT playerid, yearid, salary FROM salaries NATURAL JOIN appearances WHERE g_all = 10 OR g_batting = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.66..3927.43 rows=1 width=21) (actual time=14.344..22.402 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.006..1.854 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2841 width=20) (actual time=14.121..14.124 rows=

In [100]:
result_4eiii_cost = 3930.03
result_4eiii_timing = 25.412

In [101]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [102]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [103]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.66..3927.43 rows=1 width=21) (actual time=15.598..23.828 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.007..1.901 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2841 width=20) (actual time=15.374..15.377 rows=

In [104]:
result_4eiv_with_index_cost = 3930.03
result_4eiv_with_index_timing = 29.813

In [105]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [106]:
%%sql 
DROP INDEX g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [107]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx ON appearances(g_batting,g_all)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [108]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2883.94..3620.72 rows=1 width=21) (actual time=5.206..12.957 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.005..1.856 rows=26428 loops=1)
   ->  Hash  (cost=2827.12..2827.12 rows=2841 width=20) (actual time=4.964..4.970 rows=165

In [109]:
result_4ev_multiple_col_index_cost = 3610.64
result_4ev_multiple_col_index_timing = 15.559

In [110]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [111]:
q4e_part6 = ['C', 'I']

In [112]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

C is true. This is reflected by the lower cost and lower execution time reflected in the query plan details for with the index vs without the index. This is because the index on g_batting allowed us to find the tuples with g_batting = 10 by going directly to the pages that contained tuples with that value, as opposed to having to sequentially scan all pages for tuples that satify our constraint. Then once we are at these tuples, we need only to check our second condition, g_all. This is due to the short circut nature of the AND, that is only tuples that satify both conditions are kept, so we need not consider at all tuples that do not satify our first indexed g_batting condition. Thus A and B are false.

D, E, and F are all false. This is reflected by the cost and execution time being basically the same in the query plan details for with vs without the index. This is because although the index on g_batting would allow us to find the tuples that have g_batting = 10 more quickly than a sequential scan, in this situation we are looking for tuples with g_batting = 10 or g_all = 10, thus even if we got all the tuples that satified our first g_batting = 10, we would still have to scan all of the other tuples that had g_batting != 10 to look for g_all = 10, which is still scanning all tuples. Thus a sequential scan on all tuples was used to check for both, which is what was done in the query that didn't have an index on g_batting, thus they are the same.

I is true. This is reflected by the lower cost and lower execution time reflected in the query plan details for with the multicolumn index vs without the multicolumn index. This is because we can find all the tuples that satisfy our first column in the multicolumn index g_batting directly, but when looking for the tuples that satify the second column, g_all, instead of sequentially scanning all tuples, we can use the index because the second column has a relative ordering in the index where values that have g_all = 10 will be ordered relative to the tuples where g_batting != 10. 

In [113]:
%%sql
DROP INDEX g_batting_g_all_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [114]:
%%sql
SELECT AVG(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.


avg
2085634.053125473


In [115]:
%%sql result_5a_min << 
SELECT MIN(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [116]:
result_5a_min_query_str = "SELECT MIN(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.795..5.797 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.013..2.401 rows=26428 loops=1)
 Planning Time: 0.428 ms
 Execution Time: 5.863 ms
(4 rows)



In [117]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 4.799

In [118]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [119]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [120]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=6.607..6.609 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.006..2.531 rows=26428 loops=1)
 Planning Time: 0.478 ms
 Execution Time: 6.713 ms
(4 rows)



In [121]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 5.671

In [122]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [123]:
%%sql 
CREATE INDEX salary_idx ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [124]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.062..0.063 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.060..0.060 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.058..0.058 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.618 ms
 Execution Time: 0.116 ms
(8 rows)



In [125]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.310

In [126]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [127]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.090..5.092 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.006..1.908 rows=26428 loops=1)
 Planning Time: 0.481 ms
 Execution Time: 5.192 ms
(4 rows)



In [128]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 5.826

In [129]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [130]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [131]:
q5c = ['D']

In [132]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

D is true. This is reflected by the lower cost and execution time for the min query and the same cost and execution time for the average query with vs without the index. With the index, our table is ordered by the salary, so extreme values like min and max are very quick to find because we can just go straight to the highest or lowest index. However the index doesn't help for averages or counts because these aggregations involve all of our tuples, so each tuple must be considered, regardless of their value, so indexing them by the value has no effect. Thus A,B, and C are false.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [133]:
%%sql
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,yearid,teamid,ab
dalryab01,1884,CHN,521
hornujo01,1884,BSN,518
ansonca01,1886,CHN,504
bierblo01,1886,PH4,522
comisch01,1886,SL4,578
corkhpo01,1886,CN2,540
gleasbi01,1886,SL4,524
hankifr01,1886,NY4,522
larkihe01,1886,PH4,565
lathaar01,1886,SL4,578


In [134]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [135]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [136]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [137]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8871 width=21) (actual time=0.247..12.883 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.379 ms
 Execution Time: 13.292 ms
(5 rows)



In [138]:
result_6a_cost = 2884.05
result_6a_timing = 25.536

In [139]:
grader.check("q6aii")

q6aii results: All test cases passed!

In [140]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
26 rows affected.


Schema,Name,Type,Owner
public,allstarfull_pkey,index,jovyan
public,appearances_pkey,index,jovyan
public,awardsmanagers_pkey,index,jovyan
public,awardsplayers_pkey,index,jovyan
public,awardssharemanagers_pkey,index,jovyan
public,awardsshareplayers_pkey,index,jovyan
public,batting_pkey,index,jovyan
public,battingpost_pkey,index,jovyan
public,collegeplaying_pkey,index,jovyan
public,fielding_pkey,index,jovyan


### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [141]:
%%sql 
CLUSTER batting USING batting_pkey 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [142]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8871 width=21) (actual time=0.013..16.397 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.403 ms
 Execution Time: 16.807 ms
(5 rows)



In [143]:
result_6b_cost = 2878.05
result_6b_timing = 16.191

In [144]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [145]:
%%sql 
CREATE INDEX ab_idx ON batting(ab);
CLUSTER batting USING ab_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [146]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=101.04..1786.93 rows=8871 width=21) (actual time=0.324..1.911 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..98.83 rows=8871 width=0) (actual time=0.290..0.290 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.540 ms
 Execution Time: 2.313 ms
(7 rows)



In [147]:
result_6c_cost = 1787.68
result_6c_timing = 2.429

In [148]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [149]:
q6d = ['A', 'D']

In [150]:
grader.check("q6d")

q6d results: All test cases passed!

In [151]:
%%sql
DROP INDEX ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

A and D are true. This is reflected by the query plan details showing that clusting using ab_idx did reduce query cost and execution time. This because we were filtering by ab>500, and having our data ordered by ab allowed us to skip to indexes with a higher ab. Thus B and C are false, and I is also false.

E,F,G, and H are all false. This is shown by the query plan being the same for with the cluster by batting_pkey vs without the clustering. This is because our filter was based on the value of ab, which in each tuple is unrelated to the batting_pkey, so having our tuples sorted by the batting_pkey had no effect on the way we find the tuples with ab>500, thus we had to do a sequential scan exactly the same as without the clustering

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [152]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [153]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 5.4 ms, sys: 1.19 ms, total: 6.59 ms
Wall time: 3.75 s


[]

In [154]:
result_7a_timing = 3.81

In [155]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [156]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [157]:
%%sql 
CREATE INDEX salary_idx ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [158]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.67 ms, sys: 3.37 ms, total: 7.04 ms
Wall time: 6.58 s


[]

In [159]:
result_7b_timing = 5.81

In [160]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

With the index on on salary, our update time was longer. This is because with the index added on salary, when adding our new tuples that contain random salary values, for each tuple we must find in our table the correct spot to place it with respect to the salary value of all other tuples already present in order to maintain correct index ordering, where without the index, we could insert the new tuples anywhere into the table regardless of it's salary value.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1) Use of materialized views
If there are relations in our database that we are joining, or aggregating on, when the relations are very large, these operations can become very costly and time consuming. This can be very problematic if we are frequently doing these operations. One way to reduce cost and execution time is to use a materialized view. The materialized view can allow us to pre-compute expensive joins and aggregations, so we can reference them over and over again, without the associated cost. They also have the added benefit of being able to be refreshed when the underlying tables they are built on change. This gives them an advantage over simply making a new table. If we are using the same subquery very often in different operations, replacing the subquery with a materialized view can give a significant speed up to all operations that use that subquery. If we have a subquery that joins multiple large tables or aggregates large tables, but the result contains few rows, then the materialized view won't take up much space and is a good choice. However materialized views should not be used if the base tables change frequently as constant updates will be expensive, in this case a virtual view or simple subquery would be better.

2) Use of indexes
If we have an attribute in our table that we often use to filter by, then creating an index on that attribute can reduce cost and execution time of operations that condition on it. The index allows the optimizer to go directly to data pages that have the values of the attribute we are looking for, without having to sequentially scan all the tuples in our table, which can be especially helpful if the attribute has a very high cardinality. In the case where we are joining on this attribute, then we need only consider a much smaller subset of our data pages that contain the desired value, increasing join speed. They also allow us to compute min and max aggregations very quickly by simply going to the highest or lowest index. Sorting speed is also increased because we already have pointers that are sorted by the attributes values. All these benefits can be extended with multicolumn indexes, while still retaining all the desired index properties of the first attribute by the structuring of multicolumn indexes. However if we are inserting into our table many widely ranging values of an attribute we have an index on, then we must insert at the right spot to preserve the index integrity, which can become very costly to maintain.

3) Join Selection
The query optimizer can look at conditions not related to the attribute being joined on, and select a join based on subsequent operations. For example if we are performing a equality join on an attribute, then ordering by that attribute, the query optimizer can pick a merge-join, which will always have a sorted output, therefore skipping the need to sort the result if a hash join was used, speeding up our query. Also if our join attribute is already sorted in our tables by an index, merge join can be very fast by exploiting the index order. The optimizer can also pick a nested loop join if one of our relations is so small that the inner loop is not executed very often, increasing speed by skipping the preprocessing required by a hash join or merge join. If one table can be hashed into working memory, then hash join will be selected because preprocessing one side, then scanning and probing the hash table will usually be faster than nested loops double scan and merge sort preprocessing both relations. By using these estimates of hashing time or scanning time, the optimizer will usually always select the fastest option. However if the estimates by the optimizer are wrong, then joins like nested loop can be wrongly selected and dramatically increase cost and time.


<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [ ]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()